# **Ecommerce_Customer_Revenue_Analysis**

**PROJECT OBJECTIVE**             
Analyze e-commerce transactional data to understand sales trends, customer behavior, and product performance using SQL for querying and Python for EDA & visualization.

**Import Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

**LOAD DATASET**

In [ ]:
df = pd.read_csv('/content/ecommerce_customer_data_large.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

**DATA CLEANING**

Remove Duplicates

In [ ]:
df.drop_duplicates(inplace=True)

Convert date column

In [ ]:
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], errors='coerce')

Remove cancelled orders

In [ ]:
df = df[df['Returns'] == 0]

**Feature Engineering**

Create revenue column

In [ ]:
df['revenue'] = df['Product Price'] * df['Quantity']

In [ ]:
df['month'] = df['Purchase Date'].dt.to_period('M')

In [ ]:
df['year'] = df['Purchase Date'].dt.year

In [ ]:
df.isnull().sum()

**LOAD DATA INTO SQL (SQLite)**

In [ ]:
conn = sqlite3.connect("ecommerce.db")

# Convert 'month' column to string type as SQLite does not support Period type
df['month'] = df['month'].astype(str)

df.to_sql("orders", conn, if_exists="replace", index=False)

Monthly Revenue Trend

In [ ]:
query = """
SELECT
    strftime('%Y-%m', "Purchase Date") AS month,
    SUM(revenue) AS monthly_revenue
FROM orders
GROUP BY month
ORDER BY month;
"""
monthly_revenue = pd.read_sql(query, conn)
monthly_revenue

Top 10 Product Categories by Revenue

In [ ]:
query = """
SELECT
    "Product Category" AS product_category,
    SUM(revenue) AS total_revenue
FROM orders
GROUP BY "Product Category"
ORDER BY total_revenue DESC
LIMIT 10;
"""
top_categories = pd.read_sql(query, conn)
top_categories

Average Order Value (AOV)

In [ ]:
query = """
SELECT
    SUM(revenue) / COUNT(*) AS avg_order_value
FROM orders;
"""
pd.read_sql(query, conn)

Repeat vs One-Time Customers

In [ ]:
query = """
SELECT
    "Customer ID" AS customer_id,
    COUNT(*) AS total_orders
FROM orders
GROUP BY "Customer ID";
"""
customer_orders = pd.read_sql(query, conn)
customer_orders.head()

Rank customers by revenue within each product category.

In [ ]:
query = """
SELECT
    "Customer ID" AS customer_id,
    "Product Category" AS product_category,
    SUM(revenue) AS total_revenue,
    RANK() OVER (
        PARTITION BY "Product Category"
        ORDER BY SUM(revenue) DESC
    ) AS revenue_rank
FROM orders
GROUP BY "Customer ID", "Product Category";
"""
customer_category_rank = pd.read_sql(query, conn)
customer_category_rank.head()

Find churned customers who generated above-average revenue.

In [ ]:
query = """
SELECT
    "Customer ID" AS customer_id,
    SUM(revenue) AS total_revenue
FROM orders
WHERE churn = 1
GROUP BY "Customer ID"
HAVING total_revenue > (
    SELECT AVG(revenue) FROM orders
);
"""
churned_high_revenue_customers = pd.read_sql(query, conn)
churned_high_revenue_customers.head()

Find customers whose total revenue is greater than the average customer revenue.

In [ ]:
query = """
SELECT "Customer ID" AS customer_id
FROM orders
GROUP BY "Customer ID"
HAVING SUM(revenue) >
       (SELECT AVG(revenue) FROM orders);
"""
pd.read_sql(query, conn)

Compare returned vs non-returned revenue.

In [ ]:
query = """
SELECT
    "Product Category" AS product_category,
    SUM(CASE WHEN "Returns" = 0 THEN revenue ELSE 0 END) AS completed_revenue,
    SUM(CASE WHEN "Returns" = 1 THEN revenue ELSE 0 END) AS returned_revenue
FROM orders
GROUP BY "Product Category";
"""
returned_vs_completed_revenue = pd.read_sql(query, conn)
returned_vs_completed_revenue.head()

**CUSTOMER SEGMENTATION**         
 Segment customers based on purchase frequency.

In [ ]:
query = """
SELECT
    "Customer ID" AS customer_id,
    COUNT(*) AS total_orders,
    CASE
        WHEN COUNT(*) = 1 THEN 'One-Time'
        WHEN COUNT(*) BETWEEN 2 AND 5 THEN 'Regular'
        ELSE 'Loyal'
    END AS customer_segment
FROM orders
WHERE "Returns" = 0
GROUP BY "Customer ID";
"""
customer_segments = pd.read_sql(query, conn)
customer_segments.head()

**TOP-N PER GROUP**

Find top 3 customers by revenue in each product category.

In [ ]:
query = """
SELECT *
FROM (
    SELECT
        "Customer ID" AS customer_id,
        "Product Category" AS product_category,
        SUM(revenue) AS total_revenue,
        ROW_NUMBER() OVER (
            PARTITION BY "Product Category"
            ORDER BY SUM(revenue) DESC
        ) AS rn
    FROM orders
    GROUP BY "Customer ID", "Product Category"
)
WHERE rn <= 3;
"""
top_3_customers_by_category = pd.read_sql(query, conn)
top_3_customers_by_category.head()

**TIME-BASED ANALYSIS**
Find month-over-month revenue growth.

In [ ]:
query = """
SELECT
    month,
    monthly_revenue,
    monthly_revenue - LAG(monthly_revenue)
        OVER (ORDER BY month) AS mom_growth
FROM (
    SELECT
        strftime('%Y-%m', "Purchase Date") AS month,
        SUM(revenue) AS monthly_revenue
    FROM orders
    WHERE "Returns" = 0
    GROUP BY month
);
"""
mom_revenue_growth = pd.read_sql(query, conn)
mom_revenue_growth.head()

**PAYMENT METHOD EFFECTIVENESS**

Find payment methods with highest return rate.

In [ ]:
query = """
SELECT
    "Payment Method" AS payment_method,
    COUNT(CASE WHEN "Returns" = 1 THEN 1 END) * 1.0 / COUNT(*) AS return_rate
FROM orders
GROUP BY "Payment Method"
ORDER BY return_rate DESC;
"""
pd.read_sql(query, conn)

**DATA QUALITY VALIDATION**                
Identify inconsistent revenue records.

In [ ]:
query = """
SELECT *
FROM orders
WHERE "Total Purchase Amount" != "Product Price" * Quantity;
"""
pd.read_sql(query, conn)

**Outlier Treatment (IQR Method)**

In [ ]:
Q1 = df['Quantity'].quantile(0.25)
Q3 = df['Quantity'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['Quantity'] >= Q1 - 1.5 * IQR) &
        (df['Quantity'] <= Q3 + 1.5 * IQR)]

**EDA (Exploratory Data Analysis)**

**Monthly Revenue Trend**

In [ ]:
# Create month column from purchase date
df['month'] = pd.to_datetime(df['Purchase Date']).dt.to_period('M')

# Calculate monthly revenue
monthly_rev = (
    df.groupby('month')['revenue']
    .sum()
    .reset_index()
)


In [ ]:
plt.figure(figsize=(12,6))
monthly_rev['month'] = monthly_rev['month'].astype(str)

sns.lineplot(
    data=monthly_rev,
    x='month',
    y='revenue',
    color='green',
    marker='o',
    linewidth=2
)
plt.title("Monthly Revenue Trend", fontsize=14, fontweight='bold')
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Revenue", fontsize=12)

plt.xticks(rotation=90)
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig("monthly_revenue_trend.png", dpi=300, bbox_inches='tight')
plt.show()


**Top 10 Product Categories**

In [ ]:
plt.figure(figsize=(12, 6))

sns.barplot(
    data=top_categories,
    x='product_category',
    y='total_revenue',
    palette="viridis"
)

plt.title("Top 10 Product Categories by Revenue", fontsize=14, fontweight="bold")
plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Total Revenue", fontsize=12)

plt.xticks(rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.6)

plt.tight_layout()
plt.savefig("Top_10_Product_Categories.png", dpi=300, bbox_inches='tight')
plt.show()

**Revenue Distribution**

In [ ]:
plt.figure(figsize=(10, 5))

sns.boxplot(
    x=df['revenue'],
    color='yellowgreen',
    linewidth=1.5,
    showfliers=True
)

plt.title("Revenue Distribution with Outliers", fontsize=14, fontweight="bold")
plt.xlabel("Revenue", fontsize=12)

plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("revenue_distribution.png", dpi=300, bbox_inches='tight')
plt.show()


**Payment Method Analysis**

In [ ]:
plt.figure(figsize=(10, 6))

sns.countplot(
    data=df,
    x='Payment Method',
    palette="viridis",
    order=df['Payment Method'].value_counts().index
)

plt.title("Payment Method Distribution", fontsize=14, fontweight='bold')
plt.xlabel("Payment Method", fontsize=12)
plt.ylabel("Number of Transactions", fontsize=12)

plt.xticks(rotation=30, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("payment_method_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

**Customer Segmentation**

In [ ]:
segment_counts = customer_segments['customer_segment'].value_counts().sort_index()

plt.figure(figsize=(7,7))
plt.pie(
    segment_counts.values,
    labels=segment_counts.index,
    autopct='%1.1f%%',
    colors=sns.color_palette('viridis'),
    startangle=120,
    wedgeprops={'edgecolor':'black'}
)
plt.title("Customer Segmentation Proportion", fontsize=14, fontweight='bold')
plt.savefig("customer_segmentation.png", dpi=300, bbox_inches='tight')
plt.show()

**Segment-wise Revenue**

In [ ]:
# Calculate total revenue for each customer segment
# 1. Merge df with customer_segments to get the segment for each transaction
merged_df_for_segments = pd.merge(df, customer_segments, left_on='Customer ID', right_on='customer_id', how='inner')

# 2. Group by 'customer_segment' and sum 'revenue'
customer_summary = merged_df_for_segments.groupby('customer_segment')['revenue'].sum().reset_index()
customer_summary.columns = ['segment', 'total_revenue']

plt.figure(figsize=(8, 6))

sns.barplot(
    data=customer_summary,
    x='segment',
    y='total_revenue',
    palette='viridis'
)

plt.title("Total Revenue by Customer Segment", fontsize=14, fontweight='bold')
plt.xlabel("Customer Segment", fontsize=12)
plt.ylabel("Total Revenue", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6);

# Add total revenue labels on top of bars
# Use the already prepared customer_summary for segment_revenue
segment_revenue = customer_summary.set_index('segment')['total_revenue']
for i, v in enumerate(segment_revenue):
    plt.text(i, v + segment_revenue.max()*0.01, f'{v:,.0f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig("segment_wise_revenue.png", dpi=300, bbox_inches='tight')
plt.show()

# BUSINESS INSIGHTS


~20–30% customers generate majority revenue   
Loyal customers have higher average order value  
Few categories dominate total sales      
Digital payments show higher completion rate

# RECOMMENDATIONS
Introduce loyalty programs for repeat customers

Focus marketing on high-margin categories

Promote prepaid payment methods

Target peak months with campaigns